In [1]:
import numpy as np

In [2]:
np.random.seed(0)

In [3]:
# data = "hello world"
data = open("./data/howtogetrich.txt", "r").read()[:50]
chars = list(set(data))
vocab_size = len(chars)

chars_to_indices = {char: i for i, char in enumerate(chars)}
indices_to_chars = {i: char for i, char in enumerate(chars)}

chars_to_indices, indices_to_chars

({'S': 0,
  'N': 1,
  'a': 2,
  't': 3,
  'k': 4,
  'M': 5,
  'l': 6,
  'r': 7,
  ',': 8,
  'h': 9,
  'o': 10,
  's': 11,
  'u': 12,
  'i': 13,
  '\n': 14,
  'e': 15,
  'W': 16,
  ' ': 17,
  'n': 18,
  'y': 19},
 {0: 'S',
  1: 'N',
  2: 'a',
  3: 't',
  4: 'k',
  5: 'M',
  6: 'l',
  7: 'r',
  8: ',',
  9: 'h',
  10: 'o',
  11: 's',
  12: 'u',
  13: 'i',
  14: '\n',
  15: 'e',
  16: 'W',
  17: ' ',
  18: 'n',
  19: 'y'})

In [4]:
# RNN configs
embedding_size = vocab_size
hidden_size = 64
output_size = vocab_size

In [5]:
# Matrices
Wxh = np.random.randn(hidden_size, embedding_size) * 0.01
Whh = np.random.randn(hidden_size, hidden_size) * 0.01
bh = np.random.randn(1, hidden_size) * 0.01

Why = np.random.randn(output_size, hidden_size)
by = np.random.randn(1, output_size)

In [6]:
def softmax(z):
    e_z = np.exp(z)
    return e_z / e_z.sum(axis=1)

def rnn(x_t, h_prev):
    assert x_t.shape == (1, embedding_size)
    assert h_prev.shape == (1, hidden_size)

    h_next = np.tanh(x_t @ Wxh.T + h_prev @ Whh.T + bh)
    probs = softmax(h_next @ Why.T + by)
    
    return probs, h_next

def sample(char, n):
    # sample rnn n times starting with the first char
    x = np.zeros((1, vocab_size))
    idx = chars_to_indices[char]
    x[:, idx] = 1 # one hot encoding
    h =  np.zeros((1, hidden_size)) # hidden state
    idxes = []
    
    for i in range(n):
        probs, h = rnn(x, h) # xt, hprev -> rnn -> probs, hnext
        idx = np.random.choice(vocab_size, p=probs.ravel()) # .ravel returns a 1d array
        
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        idxes.append(idx)

    chars = "".join([indices_to_chars[i] for i in idxes])
    return chars

In [7]:
x_t = np.random.randn(1, embedding_size)
h_prev = np.random.randn(1, hidden_size)

probs, h_next = rnn(x_t, h_prev)
probs.shape, h_next.shape

((1, 20), (1, 64))

In [8]:
inp = [chars_to_indices[c] for c in data[:-1]]
target = [chars_to_indices[c] for c in data[1:]]

inp, target

([0,
  15,
  15,
  4,
  17,
  16,
  15,
  2,
  6,
  3,
  9,
  8,
  17,
  1,
  10,
  3,
  17,
  5,
  10,
  18,
  15,
  19,
  17,
  10,
  7,
  17,
  0,
  3,
  2,
  3,
  12,
  11,
  14,
  16,
  15,
  2,
  6,
  3,
  9,
  17,
  13,
  11,
  17,
  2,
  11,
  11,
  15,
  3,
  11],
 [15,
  15,
  4,
  17,
  16,
  15,
  2,
  6,
  3,
  9,
  8,
  17,
  1,
  10,
  3,
  17,
  5,
  10,
  18,
  15,
  19,
  17,
  10,
  7,
  17,
  0,
  3,
  2,
  3,
  12,
  11,
  14,
  16,
  15,
  2,
  6,
  3,
  9,
  17,
  13,
  11,
  17,
  2,
  11,
  11,
  15,
  3,
  11,
  17])

In [10]:
sample("h", 4)

'SS h'

In [11]:
lr = 1e-2
for e in range(100):
    # compute loss
    loss = 0
    xs, ps, hs = {}, {}, {}
    h = np.zeros((1, hidden_size))
    hs[-1] = h
    for t in range(len(inp)):
        idx = inp[t]
        x = np.zeros((1, vocab_size))
        x[:, idx] = 1
        probs, h = rnn(x, h)
        xs[t] = x
        ps[t] = probs # save probs (we'll use this in backprop)
        hs[t] = h

        # cross entropy
        pred = probs[0, target[t]]
        loss += -np.log(pred)

    # backprop, calculate gradients
    dL_dWxh = np.zeros_like(Wxh)
    dL_dWhh = np.zeros_like(Whh)
    dL_dbh = np.zeros_like(bh)
    dL_Why = np.zeros_like(Why)
    dL_dby = np.zeros_like(by)
    dF_dh = np.zeros((1, hidden_size))

    for t in reversed(range(len(inp))):
        dL_dz2 = np.copy(ps[t])
        target_idx = target[t]
        dL_dz2[:, target_idx] -= 1

        # print(dL_dz2.shape)

        a = hs[t] * dL_dz2.T
        # print(a.shape)

        # 2nd layer
        dL_Why += hs[t] * dL_dz2.T
        dL_dby += dL_dz2


        # 1st layer
        dh_dz1 = 1 - hs[t] ** 2
        dL_dh = dF_dh + (dL_dz2 @ Why) 

        # print((xs[t] * dh_dz1.T * dL_dh.T).shape)

        dL_dWxh += xs[t] * dh_dz1.T * dL_dh.T
        dL_dWhh +=  hs[t - 1] * dh_dz1.T * dL_dh.T
        dL_dbh += dh_dz1 * dL_dh

        # raise RuntimeError

        # dF_dh = dL_dh * (dh_dz1 @ Whh)
        dF_dh = dL_dh @ (dh_dz1 * Whh)

    # clip gradients
    for gradient in [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]:
        np.clip(gradient, -1, 1, out=gradient)
        # np.clip(gradient, -5, 5, out=gradient)

    # gradient descent
    for weights, gradient in zip([Wxh, Whh, Why, bh, by], [dL_dWxh, dL_dWhh, dL_Why, dL_dbh, dL_dby]):
        weights -= lr * gradient

    # break
    
    print(f"[Loss]: {loss}")

[Loss]: 161.90099734913815
[Loss]: 138.81123005923382
[Loss]: 118.20969234860591
[Loss]: 101.06409679720808
[Loss]: 88.75276439609502
[Loss]: 79.80701845619102
[Loss]: 71.41177973703473
[Loss]: 64.13376318393837
[Loss]: 55.223823740104606
[Loss]: 49.03456089604532
[Loss]: 43.48175530622129
[Loss]: 38.55246932739145
[Loss]: 33.02879485535942
[Loss]: 27.354741650024508
[Loss]: 23.196315842466664
[Loss]: 18.818272437090553
[Loss]: 16.689283546573847
[Loss]: 13.091161929333808
[Loss]: 11.915546077180503
[Loss]: 10.059517517927802
[Loss]: 9.544689585368982
[Loss]: 9.232571861940162
[Loss]: 8.02530953735336
[Loss]: 7.722991936946581
[Loss]: 6.780050838243331
[Loss]: 6.4746052950006145
[Loss]: 5.867295849591302
[Loss]: 5.673959068295171
[Loss]: 5.20003705883193
[Loss]: 5.127539730991191
[Loss]: 4.723859057619936
[Loss]: 4.713629903965679
[Loss]: 4.36195600796531
[Loss]: 4.372718381984839
[Loss]: 4.066042805784638
[Loss]: 4.08892969479131
[Loss]: 3.816115060604503
[Loss]: 3.849125078437209
[Lo

In [15]:
sample("S", 50)

'eek Wealth, Not Money or Status\nWealth is assets W'